<a href="https://colab.research.google.com/github/Vishnucreate/Ai_ml_notebooks/blob/main/InstructionBook1_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets huggingface_hub --quiet


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
!rm -rf /root/.cache/huggingface/datasets/institutional——institutional-books-1.0


In [ ]:
from huggingface_hub import HfApi, hf_hub_download
import pyarrow.parquet as pq

# 1) List & filter shards
api = HfApi()
files = api.list_repo_files(
    repo_id="instdin/institutional-books-1.0",
    repo_type="dataset"
)
shards = sorted(f for f in files if f.startswith("train/") and f.endswith(".parquet"))

# 2) Download only the very first shard
first = shards[0]
local_file = hf_hub_download(
    repo_id="instdin/institutional-books-1.0",
    repo_type="dataset",
    filename=first
)

# 3) Read only barcode_src and text_by_page_gen
table = pq.read_table(
    local_file,
    columns=["barcode_src", "text_by_page_gen"]
)
df = table.to_pandas()

# 4) Display the book ID and first 500 chars of page 1
book_id = df.loc[0, "barcode_src"]
page1   = df.loc[0, "text_by_page_gen"][0]  # it's a Python list of strings

print("Book ID (barcode):", book_id)
print("Page 1 snippet:\n", page1[:500], "…")


IndexError: list index out of range

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
files = api.list_repo_files(
    repo_id="instdin/institutional-books-1.0",
    repo_type="dataset"
)

# 1) Peek at the first 20 entries to see how they're named
print("First 20 files in the repo:")
for f in files[:20]:
    print(" ", f)

# 2) Now build a list of all Parquet files (no prefix filter yet)
parquets = [f for f in files if f.endswith(".parquet")]
print(f"\nFound {len(parquets)} .parquet files; e.g.:")
print(parquets[:5])

# 3) Pick the very first one
first_shard = parquets[0]
print("\nDownloading shard:", first_shard)


First 20 files in the repo:
  .gitattributes
  README.md
  data/train-00000-of-09831.parquet
  data/train-00001-of-09831.parquet
  data/train-00002-of-09831.parquet
  data/train-00003-of-09831.parquet
  data/train-00004-of-09831.parquet
  data/train-00005-of-09831.parquet
  data/train-00006-of-09831.parquet
  data/train-00007-of-09831.parquet
  data/train-00008-of-09831.parquet
  data/train-00009-of-09831.parquet
  data/train-00010-of-09831.parquet
  data/train-00011-of-09831.parquet
  data/train-00012-of-09831.parquet
  data/train-00013-of-09831.parquet
  data/train-00014-of-09831.parquet
  data/train-00015-of-09831.parquet
  data/train-00016-of-09831.parquet
  data/train-00017-of-09831.parquet

Found 9831 .parquet files; e.g.:
['data/train-00000-of-09831.parquet', 'data/train-00001-of-09831.parquet', 'data/train-00002-of-09831.parquet', 'data/train-00003-of-09831.parquet', 'data/train-00004-of-09831.parquet']



In [ ]:
from huggingface_hub import HfApi, hf_hub_download
import pyarrow.parquet as pq

api = HfApi()

# 1) List all .parquet files under data/
files = api.list_repo_files(
    repo_id="instdin/institutional-books-1.0",
    repo_type="dataset"
)
shards = sorted(f for f in files if f.startswith("data/") and f.endswith(".parquet"))

# 2) Pick the very first shard
first_shard = shards[0]  # should be 'data/train-00000-of-09831.parquet'
print("Downloading shard:", first_shard)

# 3) Download that single file
local_path = hf_hub_download(
    repo_id="instdin/institutional-books-1.0",
    repo_type="dataset",
    filename=first_shard
)

# 4) Read only the barcode and first page of text
table = pq.read_table(
    local_path,
    columns=["barcode_src", "text_by_page_gen"]
)
df = table.to_pandas()

print("Book barcode:", df.loc[0, "barcode_src"])
print("Page 1 snippet:\n", df.loc[0, "text_by_page_gen"][0][:500], "…")


Book barcode: 32044000000018
Page 1 snippet:
  …


In [ ]:
# suppose `df` is your DataFrame from the last snippet

pages = df.loc[0, "text_by_page_gen"]   # list of page-strings

# 1) Inspect the first non-empty page and its actual start:
for i, page in enumerate(pages[:5]):
    if page.strip():   # skip empty pages
        print(f"\n--- Page {i+1} (first 200 chars) ---")
        print(page[:200].replace("\n"," "))
        break

# 2) If you want more context, print more characters:
print("\n--- Page 1 (first 2000 chars) ---")
print(pages[0][:2000])

# 3) Or join the first few pages to see flow:
snippet = "\n\n".join(pages[:3])
print("\n--- Pages 1–3 combined (first 1000 chars) ---")
print(snippet[:1000])



--- Page 2 (first 200 chars) ---
GE 728 £5   HARVARD COLLEGE   VERO TAS   SCIENCE CENTER LIBRARY

--- Page 1 (first 2000 chars) ---


--- Pages 1–3 combined (first 1000 chars) ---


GE 728 £5 

HARVARD COLLEGE 

VERO TAS 

SCIENCE CENTER LIBRARY




In [ ]:
from huggingface_hub import HfApi, hf_hub_download
import pyarrow.parquet as pq

api = HfApi()

# 1) List & filter shards
files = api.list_repo_files(
    repo_id="instdin/institutional-books-1.0",
    repo_type="dataset"
)
shards = sorted(f for f in files if f.startswith("data/") and f.endswith(".parquet"))

# 2) Download the first shard *and capture its local path*
first_shard = shards[0]
local_parquet = hf_hub_download(
    repo_id="instdin/institutional-books-1.0",
    repo_type="dataset",
    filename=first_shard
)
print("Downloaded to:", local_parquet)

# 3) Read from that exact path
table = pq.read_table(
    local_parquet,
    columns=["barcode_src", "text_by_page_gen"]
)
df = table.to_pandas()

print("Book barcode:", df.loc[0, "barcode_src"])
print("Page 1 snippet:\n", df.loc[0, "text_by_page_gen"][0][:500], "…")


Downloaded to: /root/.cache/huggingface/hub/datasets--instdin--institutional-books-1.0/snapshots/d2f504a112fa057ddd2bc536188535a674a90270/data/train-00000-of-09831.parquet
Book barcode: 32044000000018
Page 1 snippet:
  …


In [ ]:
# Replace the path below with the exact one you saw in your log
!cp /root/.cache/huggingface/hub/datasets--instdin--institutional-books-1.0/snapshots/d2f504a112fa057ddd2bc536188535a674a90270/data/train-00000-of-09831.parquet .

# List it
!ls -lh train-00000-of-09831.parquet


-rw-r--r-- 1 root root 66M Jul  4 20:18 train-00000-of-09831.parquet


In [ ]:
# Path from your previous output:
shard_path = "/root/.cache/huggingface/hub/datasets--instdin--institutional-books-1.0/snapshots/d2f504a112fa057ddd2bc536188535a674a90270/data/train-00000-of-09831.parquet"

# 1) Read & extract book text
import pyarrow.parquet as pq
table = pq.read_table(shard_path, columns=["text_by_page_gen"])
df = table.to_pandas()
full_text = "\n\n".join(df.loc[0, "text_by_page_gen"])

# 2) Save to book1.txt
with open("book1.txt", "w", encoding="utf-8") as f:
    f.write(full_text)
print("book1.txt size:", !du -h book1.txt)

# 3) Now install only transformations & torch (no huggingface_hub)
!pip install --quiet transformers accelerate datasets peft bitsandbytes safetensors

# 4) Proceed with data chunking, JSONL creation, and fine-tune as before
import json, random
from datasets import Dataset

# Chunk & make examples
def chunk_text(text, max_chars=2000):
    sents = text.split('. ')
    cur, chunks = '', []
    for s in sents:
        if len(cur) + len(s) > max_chars:
            chunks.append(cur.strip() + '.')
            cur = s + '. '
        else:
            cur += s + '. '
    if cur: chunks.append(cur.strip())
    return chunks

chunks = chunk_text(full_text)
templates = ["What is the core argument of this passage?", "What philosophical insight does this convey?", "How might this idea apply today?"]
examples = [{"prompt": c + "\n\nQ: " + random.choice(templates) + "\nA:", "completion": " "} for c in chunks]

ds = Dataset.from_list(examples)

# Tokenize & train
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model

BASE_MODEL = "mistral-small"
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, load_in_8bit=True, device_map="auto")

lora_config = LoraConfig(task_type="CAUSAL_LM", r=8, lora_alpha=16, lora_dropout=0.05, target_modules=["q_proj","v_proj"])
model = get_peft_model(model, lora_config)

def preprocess(ex):
    p = tokenizer(ex["prompt"], truncation=True, max_length=1024)
    c = tokenizer(ex["completion"], truncation=True, max_length=256).input_ids
    ids = p.input_ids + c
    return {"input_ids": ids, "attention_mask": [1]*len(ids), "labels": [-100]*len(p.input_ids) + c}

train_ds = ds.map(preprocess, remove_columns=ds.column_names)

training_args = TrainingArguments(output_dir="fine_tune", per_device_train_batch_size=1,
                                  gradient_accumulation_steps=8, num_train_epochs=3,
                                  learning_rate=1e-4, fp16=True, logging_steps=10)
trainer = Trainer(model=model, args=training_args, train_dataset=train_ds, tokenizer=tokenizer)
trainer.train()


SyntaxError: invalid syntax (ipython-input-14-3247661985.py, line 13)

In [ ]:
# Cell 1 – shell
# (you only need to do this once per runtime)
!pip install --quiet transformers accelerate datasets peft bitsandbytes safetensors pyarrow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.9 MB/s eta 0:00:00


In [ ]:
# Cell 2 – Python
import pyarrow.parquet as pq
import json, random, os
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model

# 2.1) Read your cached shard
shard_path = "/root/.cache/huggingface/hub/datasets--instdin--institutional-books-1.0/" + \
             "snapshots/d2f504a112fa057ddd2bc536188535a674a90270/data/train-00000-of-09831.parquet"
table = pq.read_table(shard_path, columns=["text_by_page_gen"])
df = table.to_pandas()
full_text = "\n\n".join(df.loc[0, "text_by_page_gen"])

# 2.2) Save out the raw book
with open("book1.txt", "w", encoding="utf-8") as f:
    f.write(full_text)
print("✔️ Saved raw text to book1.txt")


✔️ Saved raw text to book1.txt


In [ ]:
!# Cell 3 – shell
!du -h book1.txt


748K	book1.txt


In [ ]:
# Cell 4 – Python
# Chunk and wrap in Q&A prompts
def chunk_text(text, max_chars=2000):
    sents = text.replace("\n"," ").split(". ")
    chunks, cur = [], ""
    for s in sents:
        if len(cur) + len(s) > max_chars:
            chunks.append(cur.strip() + ".")
            cur = s + ". "
        else:
            cur += s + ". "
    if cur: chunks.append(cur.strip())
    return chunks

templates = [
    "What is the core argument of this passage?",
    "What philosophical insight does this convey?",
    "How might this idea apply today?"
]
chunks = chunk_text(full_text)

with open("book1_qa.jsonl","w",encoding="utf-8") as out:
    for c in chunks:
        q = random.choice(templates)
        rec = {"prompt": c + "\n\nQ: " + q + "\nA:", "completion": " "}
        out.write(json.dumps(rec, ensure_ascii=False) + "\n")
print(f"✔️ Wrote {len(chunks)} examples to book1_qa.jsonl")


✔️ Wrote 402 examples to book1_qa.jsonl


In [ ]:
# Cell 4 – Python
# Chunk and wrap in Q&A prompts
def chunk_text(text, max_chars=2000):
    sents = text.replace("\n"," ").split(". ")
    chunks, cur = [], ""
    for s in sents:
        if len(cur) + len(s) > max_chars:
            chunks.append(cur.strip() + ".")
            cur = s + ". "
        else:
            cur += s + ". "
    if cur: chunks.append(cur.strip())
    return chunks

templates = [
    "What is the core argument of this passage?",
    "What philosophical insight does this convey?",
    "How might this idea apply today?"
]
chunks = chunk_text(full_text)

with open("book1_qa.jsonl","w",encoding="utf-8") as out:
    for c in chunks:
        q = random.choice(templates)
        rec = {"prompt": c + "\n\nQ: " + q + "\nA:", "completion": " "}
        out.write(json.dumps(rec, ensure_ascii=False) + "\n")
print(f"✔️ Wrote {len(chunks)} examples to book1_qa.jsonl")


✔️ Wrote 402 examples to book1_qa.jsonl


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

BASE_MODEL = "EleutherAI/gpt-neo-2.7B"     # <— now an ungated open model

# 1) Load tokenizer & model in 8-bit
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    device_map="auto"
)

# 2) Attach LoRA adapters to the attention Q/V projections
lora_cfg = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj","v_proj"]
)
model = get_peft_model(model, lora_cfg)

# 3) Load your JSONL Q&A dataset
ds = load_dataset("json", data_files="book1_qa.jsonl", split="train")

# 4) Tokenize & set labels
def preprocess(ex):
    p = tokenizer(ex["prompt"], truncation=True, max_length=1024)
    c = tokenizer(ex["completion"], truncation=True, max_length=256).input_ids
    ids = p["input_ids"] + c
    labels = [-100] * len(p["input_ids"]) + c
    return {
        "input_ids": ids,
        "attention_mask": [1] * len(ids),
        "labels": labels
    }

train_ds = ds.map(preprocess, remove_columns=ds.column_names)

# 5) Training arguments
training_args = TrainingArguments(
    output_dir="ft-book1-gptneo-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=1e-4,
    fp16=True,
    logging_steps=20,
    save_steps=200,
    save_total_limit=2
)

# 6) Trainer & launch
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    tokenizer=tokenizer
)
trainer.train()


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

In [ ]:
from datasets import load_dataset

# ❌ This will trigger the streaming code path and fail on local files:
# ds = load_dataset("json", data_files="book1_qa.jsonl", split="train")

# ✅ Instead, load it as a local file dataset, then pull out the 'train' split:
data_dict = load_dataset(
    "json",
    data_files={"train": "book1_qa.jsonl"},
    field="data",       # optional: use if your JSONL lines are wrapped in a "data" key
    cache_dir=None      # optional: put the cache somewhere else if you like
)
train_ds = data_dict["train"]
print(train_ds[0])


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset